# Basic Loading of the Kernel


To run the notebooks we recommend using Poetry and starting a shell with a virtual environment
prepared to use SK.

See [DEV_SETUP.md](../../python/DEV_SETUP.md) for more information.


In [ ]:
# Note: if using a Poetry virtual environment, do not run this cell
%pip install semantic-kernel==1.1.0

Let's define our kernel for this example.

In [ ]:
from semantic_kernel import Kernel

kernel = Kernel()

## Config

Let's get started with the necessary configuration to run Semantic Kernel. For Notebooks, we require a `.env` file with the proper settings for the model you use. You may copy the `.env.example` file from this directory and place it either in this current directory, or in the main semantic-kernel/python directory.

If interested, as you learn more about Semantic Kerenl, there are a few other ways to make sure your secrets, keys, and settings are used:

- Set the keys/secrets/endpoints as environment variables in your system.
- Manually configure the `api_key` or required parameters on either the `OpenAIChatCompletion` or `AzureChatCompletion` constructor with keyword arguments.

For now we want to keep things as simple as possible. We will be more explicit and use the `dotenv` library to load the settings.

**NOTE: Please make sure to include `GLOBAL_LLM_SERVICE` set to either OpenAI, AzureOpenAI, or HuggingFace in your .env file or environment variables. If this setting is not included, the Service will default to AzureOpenAI.**

## Option 1: using OpenAI

**Step 2**: Add your [OpenAI Key](https://openai.com/product/) key to either your environment variables or to the `.env` file in the same folder (org Id only if you have multiple orgs):

```
OPENAI_API_KEY="sk-..."
OPENAI_ORG_ID=""
OPENAI_CHAT_MODEL_ID=""
```
The environment variables names should match the names used in the `.env` file, as shown above.

If using the `.env` file, it is possible to configure the `ServiceSettings` with the `env_file_path` (absolute or relative).

```
chat_completion = OpenAIChatCompletion(service_id="test", env_file_path=<path_to_file>)
```

Use "keyword arguments" to instantiate an OpenAI Chat Completion service and add it to the kernel:

## Option 2: using Azure OpenAI

**Step 2**: Add your [Azure Open AI Service key](https://learn.microsoft.com/azure/cognitive-services/openai/quickstart?pivots=programming-language-studio) settings to either your system's environment variables or to the `.env` file in the same folder:

```
AZURE_OPENAI_API_KEY="..."
AZURE_OPENAI_ENDPOINT="https://..."
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME="..."
AZURE_OPENAI_TEXT_DEPLOYMENT_NAME="..."
AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME="..."
AZURE_OPENAI_API_VERSION="..."
```
The environment variables names should match the names used in the `.env` file, as shown above.

If using the `.env` file, please configure the `env_file_path` parameter with a valid path when creating the ChatCompletion class:

```
chat_completion = AzureChatCompletion(service_id="test", env_file_path=<path_to_file>)
```

Use "keyword arguments" to instantiate an Azure OpenAI Chat Completion service and add it to the kernel:

When using the kernel for AI requests, the kernel needs some settings like URL and credentials to the AI models.

The SDK currently supports OpenAI and Azure OpenAI, among other connectors.

If you need an Azure OpenAI key, go [here](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/quickstart?pivots=rest-api).


In [ ]:
# Make sure paths are correct for the imports

import os
import sys

notebook_dir = os.path.abspath("")
parent_dir = os.path.dirname(notebook_dir)
grandparent_dir = os.path.dirname(parent_dir)


sys.path.append(grandparent_dir)

We will load our settings and get the LLM service to use for the notebook.

In [ ]:
from dotenv import load_dotenv
from services import Service

load_dotenv()

global_llm_service = os.getenv("GLOBAL_LLM_SERVICE")
if global_llm_service is None:
    global_llm_service = "AzureOpenAI"

selectedService = Service(global_llm_service.lower())

print(f"Using service type: {selectedService}")

Let's load our settings and validate that the required ones exist.

In [ ]:
if selectedService == Service.AzureOpenAI:
    api_key = os.getenv("AZURE_OPENAI_API_KEY")
    chat_deployment_name = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
    endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    base_url = os.getenv("AZURE_OPENAI_BASE_URL")
    api_version = os.getenv("AZURE_OPENAI_API_VERSION")

    assert api_key is not None, "Azure OpenAI API key is not set"
    assert chat_deployment_name is not None, "Azure OpenAI Chat deployment name is not set"
elif selectedService == Service.OpenAI:
    api_key = os.getenv("OPENAI_API_KEY")
    chat_model_id = os.getenv("OPENAI_CHAT_MODEL_ID")
    org_id = os.getenv("OPENAI_ORG_ID")

    assert api_key is not None, "OpenAI API key is not set"
    assert chat_model_id is not None, "OpenAI Chat model ID is not set"
else:
    raise ValueError(f"The {selectedService} service is not valid in this example.")

In [ ]:
# Remove all services so that this cell can be re-run without restarting the kernel
kernel.remove_all_services()

service_id = None
if selectedService == Service.OpenAI:
    from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

    service_id = "default"
    kernel.add_service(
        OpenAIChatCompletion(
            service_id=service_id,
            ai_model_id=chat_model_id,
            org_id=org_id,
            api_key=api_key,
        ),
    )
elif selectedService == Service.AzureOpenAI:
    from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

    service_id = "default"
    kernel.add_service(
        AzureChatCompletion(
            service_id=service_id,
            api_key=api_key,
            deployment_name=chat_deployment_name,
            endpoint=endpoint,
            base_url=base_url,
            api_version=api_version,
        ),
    )

Great, now that you're familiar with setting up the Semantic Kernel, let's see [how we can use it to run prompts](02-running-prompts-from-file.ipynb).
